<a href="https://colab.research.google.com/github/agupte87/LearningPyspark/blob/master/Chapter3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
cd ..

/


In [2]:
%mkdir spark

In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [4]:
!wget -P /spark/ https://apache.osuosl.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop3.2.tgz

--2021-07-21 13:38:46--  https://apache.osuosl.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop3.2.tgz
Resolving apache.osuosl.org (apache.osuosl.org)... 64.50.236.52, 64.50.233.100, 140.211.166.134, ...
Connecting to apache.osuosl.org (apache.osuosl.org)|64.50.236.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 224762914 (214M) [application/x-gzip]
Saving to: ‘/spark/spark-3.0.3-bin-hadoop3.2.tgz’

spark-3.0.3-bin-had 100%[===================>] 214.35M  4.68MB/s    in 47s     

2021-07-21 13:39:33 (4.60 MB/s) - ‘/spark/spark-3.0.3-bin-hadoop3.2.tgz’ saved [224762914/224762914]



In [5]:
!tar xf /spark/spark-3.0.3-bin-hadoop3.2.tgz 
%mv spark-3.0.3-bin-hadoop3.2 /spark/




In [ ]:
!pip install -q findspark

In [9]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/sark/spark-3.0.3-bin-hadoop3.2"

In [10]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Exception: ignored

In [ ]:
%mkdir chapterData 

In [ ]:

%cd chapterData/
%mkdir retail-data


In [ ]:
%cd retail-data/
%mkdir by-day

In [ ]:
!wget -P content/chapterData/retail-data/by-day 'https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/by-day/2010-12-01.csv'
!wget -P content/chapterData/retail-data/by-day 'https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/by-day/2010-12-02.csv'
!wget -P content/chapterData/retail-data/by-day 'https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/by-day/2010-12-03.csv'
!wget -P content/chapterData/retail-data/by-day 'https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/by-day/2010-12-04.csv'
!wget -P content/chapterData/retail-data/by-day 'https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/by-day/2010-12-05.csv'
!wget -P content/chapterData/retail-data/by-day 'https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/retail-data/by-day/2010-12-06.csv'

# New Section

In [ ]:
staticDF = spark.read.format("csv")\
.option("inferSchema", "true")\
.option("header", "true")\
.load("/content/chapterData/retail-data/by-day/*.csv")

In [ ]:
staticDF.show(10)

In [ ]:
dayOfLargePurchase = staticDF.groupby("InvoiceDate").sum("UnitPrice").withColumnRenamed("sum(UnitPrice)", "SumOFUnitPrice")
dayOfLargePurchase.sort(dayOfLargePurchase.InvoiceDate.desc()).take(10)

In [ ]:
from pyspark.sql.functions import window , column , desc , col 
staticDataFrame = staticDF.selectExpr("CustomerID", "(UnitPrice*Quantity) as total_purchase_value", "InvoiceDate").groupBy(\
    col("CustomerId"),\
    window(col("InvoiceDate"),"1 day"))\
.sum("total_purchase_value")\
.withColumnRenamed("sum(total_purchase_value)", "total_purchase_value")\
.show(10)

